# Landscape Singlet Lens

## Landscape rear configuration

In [ ]:
import torch
import torchlensmaker as tlm

import matplotlib.pyplot as plt

def linear_magnification(object_coordinates, image_coordinates):
    T, V = object_coordinates, image_coordinates

    # Fit linear magnification with least square and compute residuals
    mag = torch.sum(T * V) / torch.sum(T**2)
    residuals = V - mag * T

    return mag, residuals
    

def plot_magnification(optics, sampling):
    """
    Compute and plot magnification data for the given optical system
    The system must compute object and image coordinates
    """

    # TODO add color_dim

    # Evaluate the optical stack
    output = optics(tlm.default_input(dim=2, dtype=torch.float64, sampling=sampling))

    # Extract object and image coordinate (called T and V)
    T = output.rays_object
    V = output.rays_image

    mag, _ = linear_magnification(T, V)

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(T.detach().numpy(), V.detach().numpy(), linestyle="none", marker="+")

    X = torch.linspace(T.min().item(), T.max().item(), 50)
    ax.plot(
        X.detach().numpy(),
        (mag * X).detach().numpy(),
        color="lightgrey",
        label=f"mag = {mag:.2f}",
    )

    ax.set_xlabel("Object coordinates")
    ax.set_ylabel("Image coordinates")
    ax.legend()

    plt.show()


In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
surface1 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(-65.)))
surface2 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(-25.)))

lens = tlm.Lens(surface1, surface2, (1.0, 1.5), outer_thickness=3)

# Build the optical sequence
optics = nn.Sequential(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=20),
    tlm.Gap(15),
    lens,
    tlm.Gap(70),
    tlm.ImagePlane(diameter=60),
)

tlm.show(optics, dim=2, end=100, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")

plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})


In [ ]:
# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"rays": 10, "object": 10},
    num_iter = 100,
)

# Render with final values
tlm.render_plt(optics, sampling={"rays": 10, "object": 5}, color_dim="object")
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

## Landscape front configuration

In [ ]:
import torch
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
shape1 = tlm.CircularArc(height=30, r=tlm.Parameter(torch.tensor(25.)))
shape2 = tlm.CircularArc(height=30, r=tlm.Parameter(torch.tensor(65.)))

lens = tlm.AsymmetricLens(shape1, shape2, (1.0, 1.5), outer_thickness=3.)

# Build the optical sequence
optics = tlm.OpticalSequence(
    tlm.ObjectAtInfinity(beam_diameter=40, angular_size=40),
    tlm.Gap(15),
    lens,
    tlm.Gap(20),
    tlm.Aperture(height=50, diameter=10),
    tlm.Gap(100),
    tlm.ImagePlane(height=100),
)

# Render the optical configuration with initial values
# Sample 10 rays per location, and 5 object locations: total = 50 rays
tlm.render_plt(optics, color_dim="object", sampling={"rays": 10, "object": 5})
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"rays": 10, "object": 10},
    num_iter = 200,
)

# Render with final values
tlm.render_plt(optics, sampling={"rays": 30, "object": 5}, color_dim="object")
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(lens)
display(part)